In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import ensemble,metrics,cross_validation

C:\Users\sharm_000\Anaconda2\envs\py36\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data=pd.read_csv('train.csv')
test_=pd.read_csv('test.csv')

In [3]:
data['val']='train'
test_['val']='test'

In [4]:
all_data=pd.concat([data,test_])

In [5]:
cat_cols=[i for i in all_data.columns if 'cat' in i]

In [6]:
new_data=pd.get_dummies(all_data,columns=cat_cols,drop_first=True)

In [7]:
new_data.shape

(1488028, 216)

In [8]:
new_data.columns

Index(['id', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'ps_calc_04',
       'ps_calc_05', 'ps_calc_06', 'ps_calc_07', 'ps_calc_08', 'ps_calc_09',
       ...
       'ps_ind_02_cat_4', 'ps_ind_04_cat_0', 'ps_ind_04_cat_1',
       'ps_ind_05_cat_0', 'ps_ind_05_cat_1', 'ps_ind_05_cat_2',
       'ps_ind_05_cat_3', 'ps_ind_05_cat_4', 'ps_ind_05_cat_5',
       'ps_ind_05_cat_6'],
      dtype='object', length=216)

In [9]:
X=[i for i in new_data.columns if i not in ['id','target','val']]
Y=['target']

In [11]:
train_data=new_data[new_data['val']=='train']
test_data=new_data[new_data['val']=='test']

In [12]:
del all_data
del data
del new_data

In [11]:
def gini(list_of_values):
    sorted_list = sorted(list(list_of_values))
    height, area = 0, 0
    for value in sorted_list:
        height += value
        area += height - value / 2.
    fair_area = height * len(list_of_values) / 2
    return (fair_area - area) / fair_area
  
def normalized_gini(y_pred, y):
    normalized_gini = gini(y_pred)/gini(y)
    return normalized_gini

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return [('gini', gini_score)]

In [12]:
K = 5
kf = cross_validation.KFold(train_data.shape[0],n_folds= K, random_state = 3228, shuffle = True)

In [13]:
xgb_preds=[]
for train_index, test_index in kf:
    print ('A')
    train_X, valid_X = train_data[X].loc[train_index], train_data[X].loc[test_index]
    train_y, valid_y = train_data[Y].loc[train_index], train_data[Y].loc[test_index]

    # params configuration also from anokas' kernel
    xgb_params = {
        'eta': 0.02,
        'max_depth': 6,
        'subsample': 0.9,
        'objective': 'binary:logistic',
        'silent': 1,
        'colsample_bytree': 0.9
    }

    d_train = xgb.DMatrix(train_X, train_y)
    d_valid = xgb.DMatrix(valid_X, valid_y)
    d_test = xgb.DMatrix(test_data[X])
    
    model = xgb.train(xgb_params, d_train, num_boost_round = 400)
                        
    xgb_pred = model.predict(d_test)
    xgb_preds.append(list(xgb_pred))

A
A
A
A
A


In [14]:
preds=[]
for i in range(len(xgb_preds[0])):
    sum=0
    for j in range(K):
        sum+=xgb_preds[j][i]
    preds.append(sum / K)

In [15]:
output = pd.DataFrame({'id': test_['id'], 'target': preds})
output.to_csv("sol_sub.csv".format(K), index=False)   